In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from pathlib import Path

In [8]:
gps_path = Path(r"C:\Users\Pedro\Outer Loop Consulting\2022-01 - TMR GSM - Documents\General\submodels\logistic_path_choice\data\compass_iot_traces")

gdf = gpd.read_feather(gps_path / "all_vertices_sample.feather")

trace_id = "YHoaaFSHtG1JNki+aTHZwA=="
# trace = gdf[gdf.trip_id==gdf.trip_id.unique()[-2]]
trace = gdf[gdf.trip_id==trace_id]
trace = trace.assign(trace_id= 1000)
trace = gpd.GeoDataFrame(trace, geometry=trace.geometry)

In [11]:
gdf = gpd.read_feather(gps_path / "origins.feather")
origin = gdf[gdf.trip_id==trace.trip_id.values[0]]

gdf = gpd.read_feather(gps_path / "destinations.feather")
dest = gdf[gdf.trip_id==trace.trip_id.values[0]]

stops = pd.concat([origin.assign(stop_id=0), dest.assign(stop_id=1)])
stops = gpd.GeoDataFrame(stops.sort_values(by=["stop_id"]), geometry="geometry")

In [5]:
from mapmatcher.skim_builder import SkimBuilder

In [6]:
%%time
from pathlib import Path
project_dir = Path(r"C:\Users\Pedro\Outer Loop Consulting\2022-01 - TMR GSM - Documents\General\submodels\basic_scenario")

network_file = project_dir / "network" / "sample_net.gpkg"
zone_file    = project_dir / "zone_system" / "zone_system_sample.gpkg"

centroids = gpd.read_file(zone_file, layer="centroids")
centroids.sort_values(by=["zone_id"], inplace=True)

zones = gpd.read_file(zone_file, layer="zone_system")

net = gpd.read_file(network_file, layer="network")

net_nodes = gpd.read_file(network_file, layer="nodes")


CPU times: total: 2.97 s
Wall time: 4.62 s


In [7]:
%%time
from mapmatcher.skim_builder import SkimBuilder

skb = SkimBuilder()
# skb.from_cache(str(project_dir))
skb.from_layers(network=net, zones=zones, network_nodes=net_nodes, centroids=centroids, cache_path=str(project_dir))
graph = skb.graph

CPU times: total: 1min 17s
Wall time: 2min 49s


In [12]:
from mapmatcher.network import Network
from mapmatcher.parameters import Parameters


mmnet = Network(graph=graph, links=net.to_crs(3857), nodes=net_nodes.to_crs(3857))
pars = Parameters()

In [13]:
from mapmatcher.trip import Trip

trip = Trip(gps_trace=trace.to_crs(3857), stops=stops.to_crs(3857), parameters=pars, network=mmnet)

In [14]:
trip.map_match(ignore_errors=True)

In [15]:
trip._stop_nodes

[310453050, 310452549]

In [16]:
from aequilibrae.paths import PathResults

res = PathResults()
res.prepare(graph)

In [17]:
%%time
res.compute_path(trip._stop_nodes[0], trip._stop_nodes[1])

: 

In [ ]:
res.links